In [ ]:
import sqlite3

conn = sqlite3.connect('yelp_dataset.db')
c = conn.cursor()

def Select(q):
    return c.execute(q)

def Update(q):
    c.execute(q)
    conn.commit()
    

In [ ]:
query1=Select("Select count(*) from Tips")

In [ ]:
query1.fetchone()

In [ ]:
query = Select("Select Reviews.user_id, Reviews.business_id, Reviews.starts, Reviews.review_id, \
Tips.tip from Reviews join Tips on \
 Reviews.user_id=Tips.user_id and Reviews.business_id=Tips.business_id")

In [ ]:
data=[]
for d in query:
    data.append(d)
    

In [ ]:
len(data)

In [ ]:
for d in data[:50]:
    print d

In [ ]:
index1=int(len(data)*0.8)
train_data=data[:index1]
test_data=data[index1:]

In [ ]:
print len(train_data)
print len(test_data)

In [ ]:
user_rating_dict={}
item_rating_dict={}

In [ ]:
for d in train_data:
    user,item,rating,review_id=d[0],d[1],d[2],d[3]
    if user not in user_rating_dict:
        user_rating_dict[user]=[[] for _ in range(5)]
    if item not in item_rating_dict:
        item_rating_dict[item]=[[] for _ in range(5)]
        
    user_rating_dict[user][int(rating)-1].append(review_id)
    item_rating_dict[item][int(rating)-1].append(review_id)    

In [ ]:
user_rating_dict.items()[:3]

In [ ]:
item_rating_dict.items()[:3]

In [ ]:
query_reviews=Select("Select review_id,review_text from Reviews")

In [ ]:
review_text_dict={}

In [ ]:
for row in query_reviews:
    review_text_dict[row[0]]=" ".join(row[1].encode('utf-8').strip().split())

In [ ]:
review_text_dict.items()[:2]

In [ ]:
user_item_review={}

In [ ]:
for d in test_data:
    user,item,rating=d[0],d[1],d[2]
    user_reveiws=user_rating_dict.get(user,[[] for _ in range(5)])[int(rating)-1]
    item_reviews=item_rating_dict.get(item,[[] for _ in range(5)])[int(rating)-1]
    reviews=user_reveiws+item_reviews
    text=[]
    for r in reviews:
        text.append(review_text_dict[r])
    if len(text)>0:
        user_item_review[(user,item)]="".join(text)

In [ ]:
len(user_item_review.keys())

In [ ]:
from sumy.parsers.plaintext import PlaintextParser 
from sumy.nlp.tokenizers import Tokenizer 
from sumy.summarizers.lex_rank import LexRankSummarizer 

In [ ]:
user_item_tips={}

In [ ]:
k=0
for (u,i) in user_item_review:
    parser = PlaintextParser.from_string(user_item_review[(u,i)],Tokenizer("english"))
    summarizer = LexRankSummarizer()
    summary = summarizer(parser.document, 1)
    user_item_tips[(u,i)]=summary

In [ ]:
len(user_item_tips.items())